# Redes Neurais Convolucionais

### Descriçao

---
1. O objetivo deste notebook é Redes Neurais Convolucionais.
2. Os dados para usar serão clonados do nosso próprio github, pela pasta dataset.
3. Vamos treinar nosso modelo para que ele aprenda a detectar doença em plantações de tomate.
---

### Dicionário


Class	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
tomato_mosaic_virus  	  										  	  |Class     | Imagem dos mosaico de vírus	                    |
Target Spot														  |Class    | Ponto alvo                         |
Bacterial_spot 		     										  |Class     | Pontos com bactéria	                |
Tomato_Yellow_Leaf_Curl_Virus  | Class | Vírus do enrolamento da folha amarela do tomate | float | nota final do ensino médio
Late_blight  | Class | Folhas queimadas
Leaf_Mold | Class | Molde de folha
Early_blight | Class | Preconceito
Spider_mites Two-spotted_spider_mite  | Class | Ácaros e outras pragas
Tomato___healthy   | Class | Saúde dos Tomates
Septoria_leaf_spot   | Class | Mancha foliar de Septoria



# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)

# Instalação de pacotes

In [ ]:
!pip install torch torchvision torchaudio pandas numpy keras

# Obtendo o dataset

In [ ]:
!git clone https://github.com/batestin1/coding_the_future_dio_redes_neurais.git #clona o repositorio
!mv coding_the_future_dio_redes_neurais/dataset /content/ #move apenas a pasta dataset para fora do diretorio
!rm -rf coding_the_future_dio_redes_neurais #exclui o restante que nao nos interessa



# Bibliotecas

In [ ]:
import numpy as np #para criação de matrizes
import os #para manipulação de folders e arquivos

from keras.models import Sequential #para iniciar nossa rede neural
from keras.layers import Conv2D #o metodo convolucional
from keras.layers import MaxPooling2D #etapa de max pooling
from keras.layers import Flatten #etapa de flatting
from keras.layers import Dense #para criacao das redes
from keras.layers import Dropout #para ajustar o overfitting
from keras.preprocessing import image #para testar o resultado
from keras.layers.experimental.preprocessing import Rescaling #para transformar RGB em 0 e 1
from keras.preprocessing.image import ImageDataGenerator # Para o pre-processamento de imagens
from tensorflow.keras.utils import plot_model #para visualizar nossa rede!
from keras.models import load_model #para salvar modelos do keras



# Construindo a RNC

In [ ]:
# Inicializando a rede
rnc = Sequential()

In [ ]:
#Criando Camadas Convolucionais

# Conv2D(32, (3, 3) O primeiro parametro (32) é o número de detectores de caracteristica. O (3,3) é filter detecter
# input_shape é o formato da imagem, os tres valores sao os pixels, sendo que o 3 sao as camadas rgb (uma vez que a imagem é colorida)
# funcao de ativacao é a Relu, ou Rectified Linear Unit.
rnc.add(Rescaling(scale=1.0/255)) #adiciono a primeira camada, um pre processamento de imagem transformando em 0 e 1 o RGB da imagem
rnc.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2))) #etapa do MaxPolling, é um mapa de caracteristica reduzida, ele reduz o tamanho da imagem sem perder as caracteristica. O parametro pool_size é o tamanho que definimos.

rnc.add(Conv2D(16, (3, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2)))

rnc.add(Conv2D(8, (3, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Flattening
# O flatten transforma a matriz em um vetor para entrar dentro das camadas densas (ocultas)

rnc.add(Flatten())

In [ ]:
# Full Connect

rnc.add(Dense(units= 128, activation = 'relu')) #esta defini a Esta é uma camada densa (totalmente conectada), o tipo mais comum de camada em redes neurais. Cada neurônio desta camada recebe entrada de todos os neurônios da camada anterior.
rnc.add(Dropout(rate = 0.5)) #Esta é uma técnica de regularização que ajuda a prevenir o overfitting. Durante o treinamento, a camada "desativa" aleatoriamente (define como zero) uma fração das unidades de entrada da camada durante cada atualização de treino.




In [ ]:
# Camada de Saida

#output layer
rnc.add(Dense(units = 10, activation = 'softmax')) #Define a função de ativação a ser usada para os neurônios desta camada. A função de ativação softmax transforma os valores de saída em uma distribuição de probabilidade sobre as classes, o que significa que a soma das probabilidades será igual a 1.

In [ ]:
# Copilando a rede
rnc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#rnc.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Pré-processamento das imagens


#### Treino

In [ ]:
train_datagen = ImageDataGenerator(
        shear_range=0.2, # realiza transformações geométricas da imagem, de forma aleatória
        zoom_range=0.2, # imagens que ele vai aplicar o zoom
        horizontal_flip=True) # o horizontal flip permite girar a imagem horizontal

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) #transforma a rescala do teste

In [ ]:
training_set = train_datagen.flow_from_directory(
        '/content/dataset/rnc/train',
        target_size=(128, 128),
        batch_size=128,
        class_mode='categorical' )

In [ ]:
label_map = (training_set.class_indices) # para visualizar as classes que ele identificou!

In [ ]:
label_map

#### Teste

In [ ]:
test_set = test_datagen.flow_from_directory(
        '/content/dataset/rnc/test',
        target_size=(128, 128),
        batch_size=128,
        class_mode='categorical')

 # Treinando A Rede Neural

In [ ]:
 rnc.fit_generator(
    training_set,
    steps_per_epoch=1,  # Ajuste conforme necessário
    epochs=4, # Tentativa com menor epoch para visualizacao em sala de aula, em casa voces podem ajustar para 10000, uma vez que convolucional é recomendado uma boa etapa de treino
    validation_data=test_set,
    validation_steps=1  # Ajuste conforme necessário
)

In [ ]:
# Esta celula esta com os epochs corretos que garantem o melhor treinamento
# rnc.fit_generator(
#        training_set,
#        steps_per_epoch=10,
#        epochs=2000,
#        validation_data=test_set,
#        validation_steps=10)



#### avaliação do modelo

In [ ]:
# Avaliando o modelo no conjunto de teste
test_loss, test_accuracy = rnc.evaluate(test_set, steps=50)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')


# Testando a performance

In [ ]:
label_map.keys()

In [ ]:


# Carregar uma imagem para fazer a previsão
img_path = '/content/dataset/rnc/test/Bacterial_spot/01a3cf3f-94c1-44d5-8972-8c509d62558e___GCREC_Bact.Sp 3396.JPG'
img = image.load_img(img_path, target_size=(128, 128))  # Use o mesmo tamanho usado para treino
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Normalizar a imagem se necessário
img_array /= 255.0  # Normalização pode variar dependendo do preprocessamento

# Fazer a previsão
prediction = rnc.predict(img_array)

# Obter o índice da classe com a maior probabilidade
predicted_class = np.argmax(prediction, axis=1)[0]

inverse_label_map = {v: k for k, v in label_map.items()} # Inverter o label_map para mapear índices às classes

predicted_class_name = inverse_label_map[prediction] # Obter o nome da classe predita

print(f"A imagem é: {predicted_class_name.replace('_',' ').replace('Tomato', '').replace(' ','')}")


# Salvando o modelo

In [ ]:
folder = 'rnc/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)

# Salva o modelo no diretório especificado
rnc.save_weights(os.path.join(folder, 'rede_neural_convolucional.h5'))

# Bonus Track

### Você conhece o [Teachable Machine](https://teachablemachine.withgoogle.com/train)?

1. Plataforma low code de treinamento de redes convolucionais
2. exporta em modelos .h5
3. Setup simples e intuitivo


# Importando os modelos

In [ ]:

model = load_model(os.path.join(folder, 'rede_neural_convolucional.h5'))